In [3]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy

# Some functions

This function plots an contour map anomaly using a provided xarray object and an axis.

In [5]:
def plotAnom(da, ax):

    #set contour intervals and colours
    #rainfall
    if any([da.name=='precAnom',da.name=='pr']):
        precContours=numpy.arange(-1.6,1.61,0.2)
        cmap='BrBG'
    #temp
    elif any([da.name=='tsAnom',da.name=='tas']):
        precContours=numpy.arange(-3.25,3.26,0.5)
        cmap='coolwarm'
    
    #do the plotting
    cs=plt.contourf(da.lon, da.lat, da.values, precContours, 
                    transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
    ax.coastlines(color='black')
    
    gl=ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle=':')
    gl.top_labels=False
    gl.left_labels=False
    
        #bottom legend
    cbar=plt.colorbar(orientation='horizontal', fraction=0.05, pad=0.05)
    
    #rainfall
    if da.name=='precAnom':
        cbar.set_label('Warm Season Anomaly (mm/day)')
    #temp
    elif da.name=='tsAnom':
        cbar.set_label('Warm Season Anomaly(C)')
    


This function loops though all the provides indeces and plots them for the date ranges 850-1850, 1850-2005 and 2005-2100

In [4]:
def multiPlotter(da, mask, indeces, filename):
    
    
    if da.year[0]<1850:
        years=[850,1850,2000,2100]
        #Decide how many rows we are going to have
        lenYears=3
    else:
        years=[1850,2000,2100]
        #Decide how many rows we are going to have
        lenYears=2
        

     
    lenIndeces=len(indeces)
    
    fig = plt.figure(figsize=(10*lenIndeces,8*lenYears))
    
    #for each row
    for iGroup in range(0,lenYears):

        #for each column
        for i in range(0,lenIndeces):
            
            #make a sub plot
            ax=fig.add_subplot(lenYears, #how many columns
                       lenIndeces, #how many rows
                       i+1+iGroup*lenIndeces, # index
                       projection=ccrs.Miller())
              #years to plot
            plotCond = mask[indeces[i]].sel(year=slice(years[iGroup],years[iGroup+1]))
            
            toPlot=da.where(plotCond).mean(['year','model']) #or 'experiment' for CESM
            
            meanLessThan=(toPlot<0)
            lessThanCount=(da.where(plotCond).mean(['year'])<0).sum('model')
            condLessThan=(lessThanCount>(0.8*len(da.model.values)))

            meanGreatThan=(toPlot>0)
            greatThanCount=(da.where(plotCond).mean(['year'])>0).sum('model')
            condGreatThan=(greatThanCount>(0.8*len(da.model.values)))

            combined=((meanLessThan&condLessThan) | (meanGreatThan&condGreatThan))

            #This is the number of events. Because dataset is well-curated, we can use the count from one position to be the count for everything
            nEvents=da.where(plotCond).isel(lon=0,lat=0).count().values
            
            plotAnom(toPlot, ax)
            
            #plt.pcolormesh(combined.lon, combined.lat, combined.values, hatch=['.'])
            
            
            
            ax.text(0.01,0.01,"n="+str(nEvents), transform=ax.transAxes, fontsize=22)
                            
            #in the first column, put a y-label
            if i==0:
                ax.text(-0.1, 0.5,  #xy relative to bottom left of axis
                        str(years[iGroup])+'-'+str(years[iGroup+1]), #yearRange
                        rotation='vertical', transform=ax.transAxes, 
                        verticalalignment='center', fontsize=30)
            
            #in the first row put a heading
            if iGroup ==0 :
                ax.set_title('Mean ' + da.name +' anomoly with ' + indeces[i] + ' event',
                             fontsize=22)
                
    #plt.savefig(filename)
    plt.show()

In [2]:
def indexPlotter(impactXr, fEventsXr, fEventsLs):
    
    axesLs=list()
    
    fig=plt.figure(figsize=(18,8))

    nIndex=len(fEventsLs)

    for i in range(0,nIndex):

        iEvent = fEventsLs[i]

        ax=fig.add_subplot(1, nIndex, i+1, projection=ccrs.Miller())

        nEvents=impactXr.where(fEventsXr[iEvent]==True).isel(lon=0,lat=0).count().values
                            
        plotAnom(
            impactXr.where(fEventsXr[iEvent]==True).mean('year'),
            ax,
        )
        ax.text(0.01,0.01,"n="+str(nEvents), transform=ax.transAxes, fontsize=22)
        
        plt.title(iEvent)
        
        axesLs.append(ax)
        
    return axesLs
        
        

In [7]:
def diffPlotter(da, mask, indeces, filename):
    
    years=[850,1850,2005,2100]

    lenIndeces=len(indeces)
    
    fig = plt.figure(figsize=(10*lenIndeces,8))
    
    #for each column
    for i in range(0,lenIndeces):
            
        #make a sub plot
        ax=fig.add_subplot(1, #how many rows
                   lenIndeces, #how many columns
                   i+1, # index
                   projection=ccrs.Miller(central_longitude=120))
          #years to plot
        plotCond1 = mask[indeces[i]].sel(year=slice(years[0],years[1]))
        plotCond2 = mask[indeces[i]].sel(year=slice(years[2],years[3]))

        toPlot=da.where(plotCond2).mean(['year','experiment']) - da.where(plotCond1).mean(['year','experiment'])

        plotAnom(toPlot, ax)

        #in the first column, put a y-label
        if i==0:
            ax.text(-0.1, 0.5,  #xy relative to bottom left of axis
                    "Difference between 2005-2100 and 850-1850 means", #yearRange
                    rotation='vertical', transform=ax.transAxes, 
                    verticalalignment='center', fontsize=30)

        ax.set_title('Difference in Mean ' + da.name +' Anomoly With ' + indeces[i] + ' Event',
                         fontsize=22)

    #plt.savefig(filename)
    plt.show()